In [28]:
#IMPORT LIBRARIES

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# DEFINE FILE NAME 


file_name = "Data set.xlsx"
file_path = os.path.join(os.getcwd(), file_name)

if not os.path.exists(file_path):
    raise FileNotFoundError(f"The file '{file_name}' does not exist in the current directory.")



# READ THE THREE SHEETS INTO SEPARATE DATAFRAMES


try:
    user_details_path = pd.read_excel(file_path, sheet_name='UserDetails.csv')
    cooking_sessions_path = pd.read_excel(file_path, sheet_name='CookingSessions.csv')
    order_details_path = pd.read_excel(file_path, sheet_name='OrderDetails.csv')
except ValueError as e:
    raise ValueError(f"Error reading sheets from the Excel file: {e}")



# HANDLE MISSING VALUES

order_details_path['Rating'].fillna(order_details_path['Rating'].mean(), inplace=True)
order_details_path.dropna(subset=['Rating'], inplace=True)


# MERGE THE DATA WITHOUT SUFFIXES

merged_data = pd.merge(cooking_sessions_path, order_details_path, on=['User ID', 'Session ID'], how='inner', suffixes=('', '_order'))
merged_data = pd.merge(merged_data, user_details_path, on='User ID', how='inner', suffixes=('', '_user'))


# DATA ANALYSIS

dish_column = [col for col in merged_data.columns if 'Dish Name' in col]

if dish_column:
    dish_column = dish_column[0]
    popular_dishes = merged_data[dish_column].value_counts()
else:
    raise KeyError("Column 'Dish Name' not found in merged_data. Check input files and column names.")

if 'Location' in merged_data.columns:
    location_trends = merged_data['Location'].value_counts()
else:
    location_trends = None

# CREATE GRAPHICAL REPRESENTATIONS


# Distribution of Ratings

plt.figure(figsize=(8, 6))
plt.hist(merged_data['Rating'], bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution of Ratings', fontsize=16)
plt.xlabel('Rating', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
rating_plot_path = 'rating_plot.png'
plt.savefig(rating_plot_path)
plt.close()

# Top 10 Dishes

plt.figure(figsize=(10, 6))
popular_dishes.head(10).plot(kind='bar', color='orange', edgecolor='black', alpha=0.8)
plt.title('Top 10 Dishes', fontsize=16)
plt.xlabel('Dish Name', fontsize=14)
plt.ylabel('Number of Orders', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
top_dishes_plot_path = 'top_dishes_plot.png'
plt.savefig(top_dishes_plot_path)
plt.close()

# Orders by Location 

if location_trends is not None:
    plt.figure(figsize=(8, 8))
    location_trends.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
    plt.title('Orders by Location', fontsize=16)
    plt.ylabel('')
    plt.tight_layout()
    location_plot_path = 'location_plot.png'
    plt.savefig(location_plot_path)
    plt.close()


# WRITE OUTPUT DATA INTO EXCEL 

output_file = 'output_data.xlsx'

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    
    merged_data.to_excel(writer, sheet_name='Merged Data', index=False)
    
    
    popular_dishes.head(10).to_excel(writer, sheet_name='Popular Dishes', header=True, index=True)
    
   
    if location_trends is not None:
        location_trends.to_excel(writer, sheet_name='Location Trends', header=True, index=True)
    
   
    workbook = writer.book
    worksheet = workbook.add_worksheet('Graphical Representation')
    writer.sheets['Graphical Representation'] = worksheet

   
    worksheet.insert_image('A1', rating_plot_path)
    worksheet.insert_image('A30', top_dishes_plot_path)
    if location_trends is not None:
        worksheet.insert_image('A60', location_plot_path)
    
   
    for sheet_name in ['Merged Data', 'Popular Dishes', 'Location Trends']:
        if sheet_name in writer.sheets:
            sheet = writer.sheets[sheet_name]
            for col_num, col in enumerate(merged_data.columns):
                col_width = max(10, len(col))
                sheet.set_column(col_num, col_num, col_width)

print(f"\nData has been written to {output_file}")



Data has been written to output_data.xlsx
